In [1]:
import socket
import json
import time
import threading
import random

# ---------------------------------------------------
# Mock AI Model Implementations
# ---------------------------------------------------

def run_depth_estimation(image_path):
    # Simulated inference latency
    time.sleep(0.12)
    return {
        "depth_map_mean": round(random.uniform(0.1, 0.9), 3),
        "model_used": "MiDaS-Small",
        "latency_ms": 120
    }

def run_upscaling(image_path):
    time.sleep(0.08)
    return {
        "upscaled_resolution": "1920x1080",
        "model_used": "Real_ESRGAN_x2",
        "latency_ms": 80
    }

def run_scene_detection(image_path):
    time.sleep(0.05)
    scenes = ["indoor", "outdoor", "portrait", "landscape", "low_light"]
    return {
        "scene": random.choice(scenes),
        "confidence": round(random.uniform(0.70, 0.98), 2),
        "latency_ms": 50
    }

def update_ai_preferences(preferences):
    return {
        "status": "stored",
        "stored_preferences": preferences
    }

# ---------------------------------------------------
# Command Router
# ---------------------------------------------------

def process_request(request):
    action = request.get("action", "")

    if action == "get_depth":
        return run_depth_estimation(request.get("image_path", ""))

    elif action == "upscale":
        return run_upscaling(request.get("image_path", ""))

    elif action == "scene_detect":
        return run_scene_detection(request.get("image_path", ""))

    elif action == "update_preferences":
        return update_ai_preferences(request.get("preferences", {}))

    elif action == "ping":
        return {"status": "alive", "service": "Pangea_AI_Module"}

    else:
        return {"error": "Unknown action or unsupported request type"}

# ---------------------------------------------------
# AI Server Setup
# ---------------------------------------------------

def start_ai_server():
    server = socket.socket()
    server.bind(("localhost", 5050))
    server.listen(1)

    print("AI Server running on port 5050...")
    client, addr = server.accept()
    print(f"Connected to client: {addr}")

    while True:
        raw = client.recv(4096).decode()
        if not raw:
            break

        request = json.loads(raw)
        print(f"Request received: {request}")

        response = process_request(request)
        print(f"Response sent: {response}")

        client.send(json.dumps(response).encode())

    client.close()
    server.close()

threading.Thread(target=start_ai_server, daemon=True).start()


In [2]:
import socket
import json

def send_request(sock, data):
    sock.send(json.dumps(data).encode())
    response = json.loads(sock.recv(4096).decode())
    print("Response:", response)
    print("-" * 60)
    return response

client = socket.socket()
client.connect(("localhost", 5050))

print("Connected to AI Server")
print("-" * 60)

# 1. Health check
send_request(client, {"action": "ping"})

# 2. Depth estimation request
send_request(client, {
    "action": "get_depth",
    "image_path": "sample_frame.jpg"
})

# 3. Upscaling request
send_request(client, {
    "action": "upscale",
    "image_path": "sample_frame.jpg"
})

# 4. Scene detection request
send_request(client, {
    "action": "scene_detect",
    "image_path": "sample_frame.jpg"
})

# 5. Update AI-side user preferences (AI Assistant)
send_request(client, {
    "action": "update_preferences",
    "preferences": {
        "preferred_brightness": 0.85,
        "preferred_color_temp": "cool",
        "hdr_tone_mapping": "dynamic",
        "auto_3d_depth": True
    }
})

client.close()


Connected to AI ServerConnected to client: ('127.0.0.1', 33142)

------------------------------------------------------------
Request received: {'action': 'ping'}
Response sent: {'status': 'alive', 'service': 'Pangea_AI_Module'}
Response: {'status': 'alive', 'service': 'Pangea_AI_Module'}
------------------------------------------------------------
Request received: {'action': 'get_depth', 'image_path': 'sample_frame.jpg'}
Response sent: {'depth_map_mean': 0.792, 'model_used': 'MiDaS-Small', 'latency_ms': 120}
Response: {'depth_map_mean': 0.792, 'model_used': 'MiDaS-Small', 'latency_ms': 120}
------------------------------------------------------------
Request received: {'action': 'upscale', 'image_path': 'sample_frame.jpg'}
Response sent: {'upscaled_resolution': '1920x1080', 'model_used': 'Real_ESRGAN_x2', 'latency_ms': 80}
Response: {'upscaled_resolution': '1920x1080', 'model_used': 'Real_ESRGAN_x2', 'latency_ms': 80}
------------------------------------------------------------
Reque